In [18]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, max_error

In [2]:
df = pd.read_pickle('/home/codespace/liebherr/Crane_2022031584 - 2023-01-17 - 2023-01-24.pkl')

In [3]:
df.isna().sum()

EngineSn                                                  0
DeltaTime_ms                                           6913
intake_manifold_air_temperature_1_°C                   7607
ambient_temperature_°C                                 7608
rail_pressure_1_bar                                    7607
wastegate_1_duty_cycle_%                               7608
engine_speed_1/min                                     7607
downstream_scr_nox_concentration_measured_ppm         15746
exhaust_brake_valve_1_actual_position_%                7608
boost_pressure_1_bar                                   7608
def_mass_flow_request_in_application_firmware_ul/s     7607
ambient_pressure_bar                                   7607
fuel_supply_pressure_1_bar                             7608
def_massflow_g/s                                       7607
engine_runtime_since_engine_start_s                    7608
battery_voltage_V                                      7607
engine_state_                           

In [4]:
df.rename(columns={'intake_manifold_air_temperature_1_°C' : 'intake_manifold_air_temperature_1',
                     'wastegate_1_duty_cycle_%' : 'wastegate_1_duty_cycle',
                     'engine_speed_1/min' : 'engine_speed_1',
                     'exhaust_brake_valve_1_actual_position_%' : 'exhaust_brake_valve_1_actual_position',
                     'def_mass_flow_request_in_application_firmware_ul/s' : 'def_mass_flow_request_in_application_firmware_ul',
                     'def_level_sensor_reading_%' : 'def_level_sensor_reading',
                     'def_tank_temperature_°C' : 'def_tank_temperature',
                     'oil_temperature_°C ' : 'oil_temperature',
                     'fuel_temperature_1_°C' : 'fuel_temperature_1',
                     'upstream_scr_temperature_1_°C' : 'upstream_scr_temperature_1',
                     'upstream_scr_temperature_1_°C' : 'upstream_scr_temperature_1',
                     'instantaneous_fuel_consumption_l/h' : 'instantaneous_fuel_consumption_l',
                     'coolant_temperature_1_°C' : 'coolant_temperature_1',
                     'final_limited_injection_quantity_demand_mg/str' : 'final_limited_injection_quantity_demand_mg'
                    }, inplace=True)

In [5]:
df.drop('downstream_scr_nox_concentration_measured_ppm', axis=1, inplace=True)
df.drop('dpf_load_value_g/l', axis=1, inplace=True)
df.drop('total_engine_running_time_s', axis=1, inplace=True)
df.drop('integrated_nox_emissions_engine_out_g/kWh', axis=1, inplace=True)

In [6]:
df.dropna(inplace=True)

In [7]:
df

,EngineSn,DeltaTime_ms,intake_manifold_air_temperature_1,ambient_temperature_°C,rail_pressure_1_bar,wastegate_1_duty_cycle,engine_speed_1,exhaust_brake_valve_1_actual_position,boost_pressure_1_bar,def_mass_flow_request_in_application_firmware_ul,...,def_tank_temperature,oil_temperature_°C,@timestamp,fuel_temperature_1,upstream_scr_temperature_1,instantaneous_fuel_consumption_l,coolant_temperature_1,vcv_1_actual_current_A,oil_pressure_bar,final_limited_injection_quantity_demand_mg
0,2022031584,329.0,14.2,3.2,1441.0,0.0,1594.0,72.2,1.46,162.0,...,4.0,93.4,2023-01-23T13:31:31.833Z,25.0,296.4,13.46,83.0,0.947,4.03,57.0
1,2022031584,347.0,14.2,3.2,1400.0,0.0,1594.0,72.5,1.46,158.3,...,4.0,93.4,2023-01-23T13:31:32.835Z,25.0,297.1,13.43,83.0,0.970,4.01,60.0
2,2022031584,329.0,14.3,3.2,1420.0,0.0,1594.0,72.5,1.46,160.1,...,4.0,93.4,2023-01-23T13:31:33.837Z,25.0,297.8,13.56,83.0,0.970,4.02,55.0
3,2022031584,339.0,14.3,3.2,1459.0,0.0,1595.0,72.0,1.47,162.0,...,4.0,93.4,2023-01-23T13:31:34.838Z,25.0,298.5,14.21,83.0,0.972,4.01,61.0
4,2022031584,333.0,14.4,3.2,1395.0,0.0,1594.0,72.0,1.47,163.7,...,4.0,93.4,2023-01-23T13:31:35.840Z,25.0,299.1,13.43,83.0,0.949,3.97,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2022031584,248.0,57.1,16.6,494.0,0.0,0.0,1.1,0.93,0.0,...,15.0,79.6,2023-01-18T13:11:58.368Z,46.7,120.2,0.00,79.2,0.298,0.00,0.0
2812,2022031584,248.0,57.1,16.6,494.0,0.0,0.0,1.1,0.93,0.0,...,15.0,79.5,2023-01-18T13:12:00.371Z,46.7,120.1,0.00,79.2,0.298,0.00,0.0
2813,2022031584,248.0,57.2,16.6,494.0,0.0,0.0,1.1,0.93,0.0,...,15.0,79.5,2023-01-18T13:11:53.349Z,46.7,120.4,0.00,79.2,0.298,0.00,0.0
2814,2022031584,244.0,57.2,16.6,494.0,0.0,0.0,1.1,0.93,0.0,...,15.0,79.5,2023-01-18T13:11:53.349Z,46.7,120.4,0.00,79.3,0.298,0.00,0.0


In [8]:
VariablesPredictives = ['rail_pressure_1_bar', 'exhaust_brake_valve_1_actual_position',
       'def_mass_flow_request_in_application_firmware_ul',
       'instantaneous_fuel_consumption_l']

VariableAPredire = ['boost_pressure_1_bar']

In [9]:
    x = df[VariablesPredictives]
    y = df[VariableAPredire]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)

In [14]:
xgboost=Pipeline([('scaler', StandardScaler()),
                    ('model', XGBRegressor(learning_rate = 0.1,
                                max_depth = 3,
                                min_child_weight=2,
                                gamma = 0,
                                colsample_bytree = 1,
                                objective='reg:squarederror',
                                n_jobs=-1))])

In [20]:
xgboost.fit(X_train, y_train)
pred_values=xgboost.predict(X_test)

print("\nXGBoost Results")
print('R2 Train Value:',metrics.r2_score(y_train, xgboost.predict(X_train)))
print('R2 Test Value:',metrics.r2_score(y_test, xgboost.predict(X_test)))
MAE = mean_absolute_error(y_test,pred_values)
MSE = mean_squared_error(y_test,pred_values)
RMSE = np.sqrt(MSE)

print('MAE:', MAE)
print('MSE', MSE)
print('RMSE', RMSE)


XGBoost Results
R2 Train Value: 0.9925781141761766
R2 Test Value: 0.9865650885739383
MAE: 0.017341569102256125
MSE 0.0008032815500980846
RMSE 0.028342222038825478
